In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

In [2]:
train = pd.read_excel('data/train_data.xlsx')
print(train.shape)
features = train.values[:,1:-1]
labels = train.values[:,-1].astype(int)
print(features[0])
ss = StandardScaler() # 标准化特征
features = ss.fit_transform(features)#(980, 14)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=10)

(980, 16)
[ 1.80e+01  0.00e+00  1.00e+00  2.00e-02 -6.00e+00 -5.00e-01  1.65e+02
  1.00e+00  7.57e+00  6.60e+00  4.39e+01  4.53e+01  4.46e+01  5.63e+02]


In [3]:
print(features.shape)

(980, 14)


In [4]:
def accuracy(test_labels, pred_labels):
    correct1 = np.sum(test_labels == pred_labels) # 完全正确
    correct2 = np.sum(abs(test_labels-pred_labels) <= 5) # 近似正确

    n = len(test_labels)
    return [float(correct1)/n, float(correct2)/n]

### LogisticRegression

In [5]:
score = 0;
for train, test in kfold.split(features, labels):
    clf = LogisticRegression(solver='lbfgs', multi_class='multinomial',max_iter=10000)
    clf.fit(features[train], labels[train])
    pred = clf.predict(features[test])
    score  += np.array(accuracy(pred, labels[test]))
print(score/10)

[0.49400437 0.86808829]


In [6]:
score = 0;
for train, test in kfold.split(features, labels):
    clf = LogisticRegressionCV(solver='lbfgs', cv=3,multi_class='multinomial',max_iter=10000)
    clf.fit(features[train], labels[train])
    pred = clf.predict(features[test])
    score  += np.array(accuracy(pred, labels[test]))
print(score/10)

[0.52948513 0.88299936]


### xgboost

In [7]:
from xgboost.sklearn import XGBClassifier

In [8]:
score1 = 0
for train, test in kfold.split(features, labels):
    clf = XGBClassifier()
    clf.fit(features[train], labels[train])
    pred = clf.predict(features[test])
    score1 += np.array(accuracy(pred, labels[test]))

print(score1/10)

[0.50338057 0.89282915]


### LightGBM

In [9]:
import lightgbm as lgb

In [10]:
params = {'application': 'regression_l2',
          'boosting': 'gbdt',
          'metric': 'rmse',
          'num_leaves': 70,
          'max_depth': 9,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.02,
          'min_child_samples': 150,
          'min_child_weight': 0.02,
          'lambda_l2': 0.0475,
          'verbosity': -1,
          'data_random_seed': 17}

# Additional parameters:
early_stop = 500
verbose_eval = 100
num_rounds = 1000

In [11]:
score = 0
for train, test in kfold.split(features, labels):
    d_train = lgb.Dataset(features[train],  label=labels[train])
    d_valid = lgb.Dataset(features[test],  label=labels[test])
    watchlist = [d_train, d_valid]
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=num_rounds,
                      valid_sets=watchlist,
                      verbose_eval=verbose_eval,
                      early_stopping_rounds=early_stop)
    pred = model.predict(features[test], num_iteration=model.best_iteration)
    score += np.array(accuracy(pred, labels[test]))
print(score/10)

Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 6.45872	valid_1's rmse: 6.72314
[200]	training's rmse: 5.16321	valid_1's rmse: 5.31237
[300]	training's rmse: 4.71131	valid_1's rmse: 4.73367
[400]	training's rmse: 4.54769	valid_1's rmse: 4.48272
[500]	training's rmse: 4.4592	valid_1's rmse: 4.38375
[600]	training's rmse: 4.4012	valid_1's rmse: 4.35738
[700]	training's rmse: 4.36063	valid_1's rmse: 4.34729
[800]	training's rmse: 4.32604	valid_1's rmse: 4.34258
[900]	training's rmse: 4.29753	valid_1's rmse: 4.34957
[1000]	training's rmse: 4.27432	valid_1's rmse: 4.35897
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 4.27432	valid_1's rmse: 4.35897
Training until validation scores don't improve for 500 rounds.
[100]	training's rmse: 6.3551	valid_1's rmse: 7.13418
[200]	training's rmse: 5.04704	valid_1's rmse: 5.91606
[300]	training's rmse: 4.57664	valid_1's rmse: 5.4663
[400]	training's rmse: 4.40913	valid_1's rmse: 5.29592
[50

### SVM

In [12]:
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn import svm

In [13]:
score1 = 0
for train, test in kfold.split(features, labels):
    clf = OneVsOneClassifier(svm.SVC(kernel='linear'))
    clf.fit(features[train], labels[train])
    pred = clf.predict(features[test])
    score1 += np.array(accuracy(pred, labels[test]))

print(score1/10)

[0.51812867 0.88540453]


In [14]:
score1 = 0
for train, test in kfold.split(features, labels):
    clf = OneVsRestClassifier(svm.SVC(kernel='linear'))
    clf.fit(features[train], labels[train])
    pred = clf.predict(features[test])
    score1 += np.array(accuracy(pred, labels[test]))

print(score1/10)

[0.31985189 0.71312416]


### 神经网络

In [15]:
from keras.utils.np_utils import to_categorical

categorical_labels = to_categorical(labels/5-1, num_classes=11)

Using TensorFlow backend.


In [16]:
print(categorical_labels.shape)

(980, 11)


In [17]:
print(categorical_labels[:10])

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [54]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras

In [55]:
def accuracy1(test_labels, pred_labels):
    correct1 = np.sum(test_labels == pred_labels) # 完全正确
    correct2 = np.sum(abs(test_labels-pred_labels) == 1) # 近似正确

    n = len(test_labels)
    return [float(correct1)/n, float(correct2+correct1)/n]

In [56]:
x_train,x_test,y_train,y_test = train_test_split(features, categorical_labels, test_size=0.1)
y_test = np.argmax(y_test, axis = 1)

In [212]:
model = Sequential()
model.add(Dense(units=32, input_dim=14))
model.add(Activation('relu'))
model.add(Dense(units=64))
model.add(Activation('relu'))
model.add(Dense(units=128))
model.add(Activation('relu'))
model.add(Dense(units=11))
model.add(Activation('softmax'))

In [213]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(epsilon=1e-8,lr=0.07), metrics=['accuracy'])

In [214]:
model.fit(x=x_train , y=y_train, batch_size=64, epochs=80 )

Epoch 1/80
882/882 [==============================] - 3s 3ms/step - loss: 2.4485 - acc: 0.2018
Epoch 2/80
882/882 [==============================] - 0s 57us/step - loss: 1.9910 - acc: 0.2392
Epoch 3/80
882/882 [==============================] - 0s 74us/step - loss: 1.7122 - acc: 0.3220
Epoch 4/80
882/882 [==============================] - 0s 59us/step - loss: 1.5168 - acc: 0.3923
Epoch 5/80
882/882 [==============================] - 0s 50us/step - loss: 1.5025 - acc: 0.3798
Epoch 6/80
882/882 [==============================] - 0s 57us/step - loss: 1.4993 - acc: 0.3844
Epoch 7/80
882/882 [==============================] - 0s 52us/step - loss: 1.3860 - acc: 0.4161
Epoch 8/80
882/882 [==============================] - 0s 53us/step - loss: 1.3083 - acc: 0.4399
Epoch 9/80
882/882 [==============================] - 0s 49us/step - loss: 1.3024 - acc: 0.4490
Epoch 10/80
882/882 [==============================] - 0s 46us/step - loss: 1.2703 - acc: 0.4615
Epoch 11/80
882/882 [===================

In [215]:
y = model.predict(x_test)
y = np.argmax(y,axis = 1)

In [216]:
ac = accuracy1(y_test, y)
print(ac)

[0.5204081632653061, 0.9081632653061225]


lr: 0.03 [0.42857142857142855, 0.8061224489795918]
lr=0.039 [0.35714285714285715, 0.8163265306122449]
lr: 0.04 [0.4489795918367347, 0.8877551020408163]
lr=0.041 [0.45918367346938777, 0.8775510204081632]
lr=0.042 [0.4387755102040816, 0.8673469387755102]
lr=0.045 [0.4387755102040816, 0.8571428571428571]
lr: 0.05 [0.46938775510204084, 0.8775510204081632]
lr: 0.06 [0.41836734693877553, 0.8877551020408163]
lr=0.065 [0.42857142857142855, 0.8877551020408163]
lr=0.067 [0.3673469387755102, 0.826530612244898]
lr=0.069 [0.5612244897959183, 0.9081632653061225]
lr=0.0695 [0.5204081632653061, 0.8775510204081632]
lr: 0.07 [0.47959183673469385, 0.9285714285714286](100) 
        [0.3979591836734694, 0.7653061224489796](110) 
        [0.5102040816326531, 0.8979591836734694](120) 
        [0.47959183673469385, 0.8979591836734694](80) 
        [0.40816326530612246, 0.9489795918367347](52) 
        [0.3673469387755102, 0.8877551020408163](20)
        [0.46938775510204084, 0.9387755102040817](30)
        [0.47959183673469385, 0.9081632653061225](35)
        [0.5510204081632653, 0.9081632653061225] (40)
lr=0.0705 [0.4387755102040816, 0.8877551020408163]
lr=0.071 [0.5102040816326531, 0.8877551020408163]
lr: 0.08 [0.4897959183673469, 0.8673469387755102]
lr: 0.01 [0.3979591836734694, 0.826530612244898]
lr: 0.001 [0.5, 0.8469387755102041]
lr: 0.005 [0.4489795918367347, 0.8571428571428571]

In [217]:
def train_test_model(lr, epochs, batch_size):
    score1 = 0
    i =0
    for train, test in kfold.split(features, labels):
        train_labels = to_categorical(labels[train]/5-1, num_classes=11)
        test_labels = to_categorical(labels[test]/5-1, num_classes=11)

        model = Sequential()
        model.add(Dense(units=32, input_dim=14))
        model.add(Activation('relu'))
        model.add(Dense(units=64))
        model.add(Activation('relu'))
        model.add(Dense(units=128))
        model.add(Activation('relu'))
        model.add(Dense(units=11))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(epsilon=1e-8,lr=lr), metrics=['accuracy'])
        model.fit(x=features[train] , y=train_labels, batch_size=batch_size, epochs=epochs )
        pred = model.predict(features[test])
        test_labels = np.argmax(test_labels,axis = 1)  
        pred = np.argmax(pred,axis = 1)
        print(pred)
        print(test_labels)
        score = np.array(accuracy1(pred, test_labels))
        print(score)
        score1 += np.array(accuracy1(pred, test_labels))

    print(score1/10)

In [218]:
train_test_model(lr=0.07, epochs=30, batch_size=64)

Epoch 1/30
877/877 [==============================] - 3s 3ms/step - loss: 2.2850 - acc: 0.2372
Epoch 2/30
877/877 [==============================] - 0s 56us/step - loss: 1.7006 - acc: 0.3398
Epoch 3/30
877/877 [==============================] - 0s 46us/step - loss: 1.5419 - acc: 0.3820
Epoch 4/30
877/877 [==============================] - 0s 68us/step - loss: 1.4176 - acc: 0.4276
Epoch 5/30
877/877 [==============================] - 0s 56us/step - loss: 1.4063 - acc: 0.4322
Epoch 6/30
877/877 [==============================] - 0s 51us/step - loss: 1.4035 - acc: 0.4390
Epoch 7/30
877/877 [==============================] - 0s 54us/step - loss: 1.3153 - acc: 0.4550
Epoch 8/30
877/877 [==============================] - 0s 55us/step - loss: 1.3315 - acc: 0.4652
Epoch 9/30
877/877 [==============================] - 0s 64us/step - loss: 1.3052 - acc: 0.4823
Epoch 10/30
877/877 [==============================] - 0s 75us/step - loss: 1.2506 - acc: 0.4709
Epoch 11/30
877/877 [===================

882/882 [==============================] - 0s 101us/step - loss: 1.0985 - acc: 0.5340
Epoch 27/30
882/882 [==============================] - 0s 105us/step - loss: 1.0854 - acc: 0.5351
Epoch 28/30
882/882 [==============================] - 0s 121us/step - loss: 1.1286 - acc: 0.5204
Epoch 29/30
882/882 [==============================] - 0s 94us/step - loss: 1.1190 - acc: 0.5295
Epoch 30/30
882/882 [==============================] - 0s 76us/step - loss: 1.0994 - acc: 0.5317
[2 3 2 6 9 4 2 4 2 5 5 2 3 4 5 3 2 4 3 2 5 3 3 5 2 3 3 5 3 7 4 5 2 4 5 2 2
 3 3 5 3 3 3 4 2 6 5 2 2 1 5 3 4 5 3 5 2 4 2 4 2 4 3 3 4 2 8 3 5 4 5 4 5 2
 1 4 4 3 2 3 5 3 5 5 4 5 5 7 1 5 3 7 2 2 2 7 4 3]
[ 1  3  2  6  7  4  2  4  2  6  5  2  3  2  4  3  1  4  3  2  6  2  3  8
  2  4  3  5  3  7  4  4  3  5  4  2  2  2  3  8  3  3  4  4  2  6  5  2
  1  2  7  2  4  6  4  6  3  3  1  4  0  5  2  2  4  1  7  3  5  4  9  4
  4  0  1  4  5  3  2  4  5  3  5  5  4  7  4 10  1  6  3  9  3  2  2  8
  3  0]
[0.52040816 0.87755102]


888/888 [==============================] - 0s 57us/step - loss: 1.1712 - acc: 0.5214
Epoch 19/30
888/888 [==============================] - 0s 59us/step - loss: 1.1504 - acc: 0.5169
Epoch 20/30
888/888 [==============================] - 0s 56us/step - loss: 1.1759 - acc: 0.5023
Epoch 21/30
888/888 [==============================] - 0s 51us/step - loss: 1.1466 - acc: 0.5113
Epoch 22/30
888/888 [==============================] - 0s 48us/step - loss: 1.1561 - acc: 0.5248
Epoch 23/30
888/888 [==============================] - 0s 47us/step - loss: 1.1628 - acc: 0.5236
Epoch 24/30
888/888 [==============================] - 0s 52us/step - loss: 1.2400 - acc: 0.4696
Epoch 25/30
888/888 [==============================] - 0s 47us/step - loss: 1.2047 - acc: 0.5045
Epoch 26/30
888/888 [==============================] - 0s 48us/step - loss: 1.1581 - acc: 0.5090
Epoch 27/30
888/888 [==============================] - 0s 48us/step - loss: 1.1569 - acc: 0.5270
Epoch 28/30
888/888 [=====================

In [224]:
train_test_model(lr=0.07, epochs=30, batch_size=128)

Epoch 1/30
877/877 [==============================] - 6s 7ms/step - loss: 3.3751 - acc: 0.1824
Epoch 2/30
877/877 [==============================] - 0s 62us/step - loss: 2.1312 - acc: 0.1904
Epoch 3/30
877/877 [==============================] - 0s 68us/step - loss: 2.0875 - acc: 0.2189
Epoch 4/30
877/877 [==============================] - 0s 60us/step - loss: 2.0196 - acc: 0.2052
Epoch 5/30
877/877 [==============================] - 0s 65us/step - loss: 1.8853 - acc: 0.2816
Epoch 6/30
877/877 [==============================] - 0s 76us/step - loss: 1.6781 - acc: 0.3478
Epoch 7/30
877/877 [==============================] - 0s 63us/step - loss: 1.5225 - acc: 0.3786
Epoch 8/30
877/877 [==============================] - 0s 71us/step - loss: 1.4566 - acc: 0.4242
Epoch 9/30
877/877 [==============================] - 0s 46us/step - loss: 1.4503 - acc: 0.4185
Epoch 10/30
877/877 [==============================] - 0s 68us/step - loss: 1.3880 - acc: 0.4105
Epoch 11/30
877/877 [===================

882/882 [==============================] - 0s 61us/step - loss: 1.0833 - acc: 0.5340
Epoch 26/30
882/882 [==============================] - 0s 40us/step - loss: 1.0699 - acc: 0.5227
Epoch 27/30
882/882 [==============================] - 0s 32us/step - loss: 1.0578 - acc: 0.5397
Epoch 28/30
882/882 [==============================] - 0s 37us/step - loss: 1.0647 - acc: 0.5408
Epoch 29/30
882/882 [==============================] - 0s 40us/step - loss: 1.0583 - acc: 0.5420
Epoch 30/30
882/882 [==============================] - 0s 35us/step - loss: 1.1147 - acc: 0.4989
[2 4 2 5 7 3 2 4 2 5 5 2 3 4 4 3 2 4 3 2 6 3 2 6 2 4 3 5 2 7 4 4 3 4 5 2 2
 3 2 5 3 3 4 3 2 5 4 2 2 2 5 2 4 7 4 5 2 4 2 4 2 4 3 3 4 2 7 2 5 4 7 4 5 2
 2 4 4 3 2 4 5 2 5 5 4 5 5 7 2 5 3 7 2 2 2 5 4 4]
[ 1  3  2  6  7  4  2  4  2  6  5  2  3  2  4  3  1  4  3  2  6  2  3  8
  2  4  3  5  3  7  4  4  3  5  4  2  2  2  3  8  3  3  4  4  2  6  5  2
  1  2  7  2  4  6  4  6  3  3  1  4  0  5  2  2  4  1  7  3  5  4  9  4
  4  0  1  

888/888 [==============================] - 0s 50us/step - loss: 2.0858 - acc: 0.2151
Epoch 18/30
888/888 [==============================] - 0s 52us/step - loss: 2.0843 - acc: 0.2151
Epoch 19/30
888/888 [==============================] - 0s 53us/step - loss: 2.0843 - acc: 0.2128
Epoch 20/30
888/888 [==============================] - 0s 64us/step - loss: 2.0834 - acc: 0.2151
Epoch 21/30
888/888 [==============================] - 0s 109us/step - loss: 2.0843 - acc: 0.2151
Epoch 22/30
888/888 [==============================] - 0s 92us/step - loss: 2.0849 - acc: 0.2151
Epoch 23/30
888/888 [==============================] - 0s 64us/step - loss: 2.0841 - acc: 0.2151
Epoch 24/30
888/888 [==============================] - 0s 43us/step - loss: 2.0858 - acc: 0.2151
Epoch 25/30
888/888 [==============================] - 0s 53us/step - loss: 2.0841 - acc: 0.2151
Epoch 26/30
888/888 [==============================] - 0s 50us/step - loss: 2.0846 - acc: 0.2151
Epoch 27/30
888/888 [====================

In [219]:
train_test_model(lr=0.07, epochs=40, batch_size=64)

Epoch 1/40
877/877 [==============================] - 3s 4ms/step - loss: 2.8700 - acc: 0.2018
Epoch 2/40
877/877 [==============================] - 0s 59us/step - loss: 2.0046 - acc: 0.2452
Epoch 3/40
877/877 [==============================] - 0s 58us/step - loss: 1.7289 - acc: 0.3523
Epoch 4/40
877/877 [==============================] - 0s 70us/step - loss: 1.6110 - acc: 0.3375
Epoch 5/40
877/877 [==============================] - 0s 76us/step - loss: 1.4692 - acc: 0.3911
Epoch 6/40
877/877 [==============================] - 0s 73us/step - loss: 1.3999 - acc: 0.4367
Epoch 7/40
877/877 [==============================] - 0s 63us/step - loss: 1.3310 - acc: 0.4629
Epoch 8/40
877/877 [==============================] - 0s 67us/step - loss: 1.3056 - acc: 0.4709
Epoch 9/40
877/877 [==============================] - 0s 75us/step - loss: 1.2874 - acc: 0.4778
Epoch 10/40
877/877 [==============================] - 0s 65us/step - loss: 1.2448 - acc: 0.4949
Epoch 11/40
877/877 [===================

877/877 [==============================] - 0s 50us/step - loss: 2.0761 - acc: 0.2041
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[ 4  4  3  4  4  2  5  5  4  3  4  8  0  2  3  2  6  3  0  1  6  2  4  6
  2  4  5  1  4  4  4  2  3  4  2  3  0  4  7  8  2  7  3  0  3  3  4  1
  3  6  5  7  4  2  9  5  1  3  4  3  2  7  5  5  2  2  7  1  4  6 10  7
  5  3  3  3  3  2 10  3  3  1  4  2  4  1  3  2  2  2  5  8  5  6  9  4
  2  2  3  6  4  2  4]
[0.19417476 0.45631068]
Epoch 1/40
880/880 [==============================] - 4s 4ms/step - loss: 2.2627 - acc: 0.2557
Epoch 2/40
880/880 [==============================] - 0s 42us/step - loss: 1.5252 - acc: 0.3966
Epoch 3/40
880/880 [==============================] - 0s 48us/step - loss: 1.3173 - acc: 0.4750
Epoch 4/40
880/880 [==============================] - 0s 42us/step - loss: 1.28

882/882 [==============================] - 0s 43us/step - loss: 1.1413 - acc: 0.5238
Epoch 26/40
882/882 [==============================] - 0s 66us/step - loss: 1.1852 - acc: 0.5238
Epoch 27/40
882/882 [==============================] - 0s 44us/step - loss: 1.1559 - acc: 0.4898
Epoch 28/40
882/882 [==============================] - 0s 50us/step - loss: 1.1576 - acc: 0.5420
Epoch 29/40
882/882 [==============================] - 0s 49us/step - loss: 1.2009 - acc: 0.4898
Epoch 30/40
882/882 [==============================] - 0s 51us/step - loss: 1.1370 - acc: 0.5193
Epoch 31/40
882/882 [==============================] - 0s 52us/step - loss: 1.0745 - acc: 0.5465
Epoch 32/40
882/882 [==============================] - 0s 53us/step - loss: 1.1383 - acc: 0.5329
Epoch 33/40
882/882 [==============================] - 0s 53us/step - loss: 1.1258 - acc: 0.5125
Epoch 34/40
882/882 [==============================] - 0s 56us/step - loss: 1.0991 - acc: 0.5454
Epoch 35/40
882/882 [=====================

884/884 [==============================] - 0s 42us/step - loss: 1.1460 - acc: 0.5226
Epoch 20/40
884/884 [==============================] - 0s 42us/step - loss: 1.2004 - acc: 0.4853
Epoch 21/40
884/884 [==============================] - 0s 43us/step - loss: 1.0939 - acc: 0.5317
Epoch 22/40
884/884 [==============================] - 0s 42us/step - loss: 1.1517 - acc: 0.5147
Epoch 23/40
884/884 [==============================] - 0s 42us/step - loss: 1.0754 - acc: 0.5509
Epoch 24/40
884/884 [==============================] - 0s 42us/step - loss: 1.1381 - acc: 0.4989
Epoch 25/40
884/884 [==============================] - 0s 42us/step - loss: 1.1039 - acc: 0.5181
Epoch 26/40
884/884 [==============================] - 0s 43us/step - loss: 1.0395 - acc: 0.5656
Epoch 27/40
884/884 [==============================] - 0s 44us/step - loss: 1.0872 - acc: 0.5690
Epoch 28/40
884/884 [==============================] - 0s 44us/step - loss: 1.0882 - acc: 0.5464
Epoch 29/40
884/884 [=====================

In [220]:
train_test_model(lr=0.07, epochs=50, batch_size=64)

Epoch 1/50
877/877 [==============================] - 4s 5ms/step - loss: 2.2105 - acc: 0.2360
Epoch 2/50
877/877 [==============================] - 0s 51us/step - loss: 1.5955 - acc: 0.3558
Epoch 3/50
877/877 [==============================] - 0s 51us/step - loss: 1.5331 - acc: 0.3831
Epoch 4/50
877/877 [==============================] - 0s 60us/step - loss: 1.3973 - acc: 0.4379
Epoch 5/50
877/877 [==============================] - 0s 56us/step - loss: 1.3197 - acc: 0.4322
Epoch 6/50
877/877 [==============================] - 0s 51us/step - loss: 1.4015 - acc: 0.4344
Epoch 7/50
877/877 [==============================] - 0s 51us/step - loss: 1.3360 - acc: 0.4436
Epoch 8/50
877/877 [==============================] - 0s 50us/step - loss: 1.2499 - acc: 0.4743
Epoch 9/50
877/877 [==============================] - 0s 51us/step - loss: 1.2490 - acc: 0.4971
Epoch 10/50
877/877 [==============================] - 0s 56us/step - loss: 1.2932 - acc: 0.4698
Epoch 11/50
877/877 [===================

877/877 [==============================] - 0s 49us/step - loss: 1.1120 - acc: 0.5450
Epoch 31/50
877/877 [==============================] - 0s 65us/step - loss: 1.0779 - acc: 0.5371
Epoch 32/50
877/877 [==============================] - 0s 76us/step - loss: 1.1081 - acc: 0.5234
Epoch 33/50
877/877 [==============================] - 0s 70us/step - loss: 1.1350 - acc: 0.5234
Epoch 34/50
877/877 [==============================] - 0s 74us/step - loss: 1.0988 - acc: 0.5302
Epoch 35/50
877/877 [==============================] - 0s 79us/step - loss: 1.0932 - acc: 0.5564
Epoch 36/50
877/877 [==============================] - 0s 80us/step - loss: 1.0698 - acc: 0.5519
Epoch 37/50
877/877 [==============================] - 0s 51us/step - loss: 1.1099 - acc: 0.5268
Epoch 38/50
877/877 [==============================] - 0s 50us/step - loss: 1.1183 - acc: 0.5211
Epoch 39/50
877/877 [==============================] - 0s 52us/step - loss: 1.1319 - acc: 0.5211
Epoch 40/50
877/877 [=====================

882/882 [==============================] - 0s 53us/step - loss: 1.0372 - acc: 0.5567
Epoch 33/50
882/882 [==============================] - 0s 69us/step - loss: 1.1055 - acc: 0.5227
Epoch 34/50
882/882 [==============================] - 0s 78us/step - loss: 1.0608 - acc: 0.5544
Epoch 35/50
882/882 [==============================] - 0s 92us/step - loss: 1.0518 - acc: 0.5828
Epoch 36/50
882/882 [==============================] - 0s 85us/step - loss: 1.0604 - acc: 0.5601
Epoch 37/50
882/882 [==============================] - 0s 74us/step - loss: 1.0924 - acc: 0.5340
Epoch 38/50
882/882 [==============================] - 0s 69us/step - loss: 1.0694 - acc: 0.5499
Epoch 39/50
882/882 [==============================] - 0s 70us/step - loss: 1.0942 - acc: 0.5317
Epoch 40/50
882/882 [==============================] - 0s 60us/step - loss: 1.0951 - acc: 0.5442
Epoch 41/50
882/882 [==============================] - 0s 59us/step - loss: 1.0936 - acc: 0.5385
Epoch 42/50
882/882 [=====================

884/884 [==============================] - 0s 52us/step - loss: 1.1046 - acc: 0.5305
Epoch 34/50
884/884 [==============================] - 0s 64us/step - loss: 1.1303 - acc: 0.5147
Epoch 35/50
884/884 [==============================] - 0s 57us/step - loss: 1.1158 - acc: 0.5328
Epoch 36/50
884/884 [==============================] - 0s 55us/step - loss: 1.1024 - acc: 0.5283
Epoch 37/50
884/884 [==============================] - 0s 54us/step - loss: 1.0969 - acc: 0.5441
Epoch 38/50
884/884 [==============================] - 0s 55us/step - loss: 1.1029 - acc: 0.5283
Epoch 39/50
884/884 [==============================] - 0s 57us/step - loss: 1.1007 - acc: 0.5385
Epoch 40/50
884/884 [==============================] - 0s 57us/step - loss: 1.0713 - acc: 0.5464
Epoch 41/50
884/884 [==============================] - 0s 57us/step - loss: 1.0698 - acc: 0.5667
Epoch 42/50
884/884 [==============================] - 0s 60us/step - loss: 1.0864 - acc: 0.5452
Epoch 43/50
884/884 [=====================

In [221]:
train_test_model(lr=0.07, epochs=80, batch_size=64)

Epoch 1/80
877/877 [==============================] - 4s 5ms/step - loss: 2.2557 - acc: 0.2600
Epoch 2/80
877/877 [==============================] - 0s 75us/step - loss: 1.5737 - acc: 0.3569
Epoch 3/80
877/877 [==============================] - 0s 84us/step - loss: 1.4751 - acc: 0.4173
Epoch 4/80
877/877 [==============================] - 0s 101us/step - loss: 1.3753 - acc: 0.4504
Epoch 5/80
877/877 [==============================] - 0s 84us/step - loss: 1.2883 - acc: 0.4835
Epoch 6/80
877/877 [==============================] - 0s 81us/step - loss: 1.2931 - acc: 0.4686
Epoch 7/80
877/877 [==============================] - 0s 80us/step - loss: 1.2545 - acc: 0.4869
Epoch 8/80
877/877 [==============================] - 0s 82us/step - loss: 1.2371 - acc: 0.5029
Epoch 9/80
877/877 [==============================] - 0s 79us/step - loss: 1.1888 - acc: 0.5051
Epoch 10/80
877/877 [==============================] - 0s 83us/step - loss: 1.1783 - acc: 0.5006
Epoch 11/80
877/877 [==================

Epoch 1/80
877/877 [==============================] - 4s 5ms/step - loss: 2.0768 - acc: 0.2531
Epoch 2/80
877/877 [==============================] - 0s 75us/step - loss: 1.4628 - acc: 0.4162
Epoch 3/80
877/877 [==============================] - 0s 59us/step - loss: 1.3673 - acc: 0.4607
Epoch 4/80
877/877 [==============================] - 0s 58us/step - loss: 1.2508 - acc: 0.4572
Epoch 5/80
877/877 [==============================] - 0s 56us/step - loss: 1.2217 - acc: 0.4846
Epoch 6/80
877/877 [==============================] - 0s 56us/step - loss: 1.1705 - acc: 0.5051
Epoch 7/80
877/877 [==============================] - 0s 57us/step - loss: 1.1918 - acc: 0.5222
Epoch 8/80
877/877 [==============================] - 0s 55us/step - loss: 1.1887 - acc: 0.5165
Epoch 9/80
877/877 [==============================] - 0s 59us/step - loss: 1.1620 - acc: 0.5234
Epoch 10/80
877/877 [==============================] - 0s 64us/step - loss: 1.1925 - acc: 0.5200
Epoch 11/80
877/877 [===================

Epoch 1/80
880/880 [==============================] - 4s 5ms/step - loss: 2.0742 - acc: 0.2682
Epoch 2/80
880/880 [==============================] - 0s 50us/step - loss: 1.4422 - acc: 0.4318
Epoch 3/80
880/880 [==============================] - 0s 49us/step - loss: 1.3568 - acc: 0.4420
Epoch 4/80
880/880 [==============================] - 0s 49us/step - loss: 1.2746 - acc: 0.4875
Epoch 5/80
880/880 [==============================] - 0s 50us/step - loss: 1.2605 - acc: 0.4636
Epoch 6/80
880/880 [==============================] - 0s 52us/step - loss: 1.2707 - acc: 0.4659
Epoch 7/80
880/880 [==============================] - 0s 49us/step - loss: 1.2407 - acc: 0.5045
Epoch 8/80
880/880 [==============================] - 0s 59us/step - loss: 1.2660 - acc: 0.4705
Epoch 9/80
880/880 [==============================] - 0s 55us/step - loss: 1.2428 - acc: 0.4784
Epoch 10/80
880/880 [==============================] - 0s 67us/step - loss: 1.2623 - acc: 0.4557
Epoch 11/80
880/880 [===================

Epoch 1/80
881/881 [==============================] - 4s 5ms/step - loss: 2.1285 - acc: 0.2236
Epoch 2/80
881/881 [==============================] - 0s 61us/step - loss: 1.5513 - acc: 0.3848
Epoch 3/80
881/881 [==============================] - 0s 47us/step - loss: 1.4891 - acc: 0.4268
Epoch 4/80
881/881 [==============================] - 0s 51us/step - loss: 1.3073 - acc: 0.4552
Epoch 5/80
881/881 [==============================] - 0s 52us/step - loss: 1.3226 - acc: 0.4756
Epoch 6/80
881/881 [==============================] - 0s 60us/step - loss: 1.2174 - acc: 0.4994
Epoch 7/80
881/881 [==============================] - 0s 51us/step - loss: 1.2343 - acc: 0.4835
Epoch 8/80
881/881 [==============================] - 0s 51us/step - loss: 1.1852 - acc: 0.5062
Epoch 9/80
881/881 [==============================] - 0s 66us/step - loss: 1.1807 - acc: 0.5142
Epoch 10/80
881/881 [==============================] - 0s 58us/step - loss: 1.1717 - acc: 0.5017
Epoch 11/80
881/881 [===================

Epoch 1/80
882/882 [==============================] - 4s 5ms/step - loss: 2.2890 - acc: 0.2245
Epoch 2/80
882/882 [==============================] - 0s 54us/step - loss: 1.6826 - acc: 0.3356
Epoch 3/80
882/882 [==============================] - 0s 43us/step - loss: 1.4375 - acc: 0.4252
Epoch 4/80
882/882 [==============================] - 0s 44us/step - loss: 1.3220 - acc: 0.4376
Epoch 5/80
882/882 [==============================] - 0s 77us/step - loss: 1.2152 - acc: 0.4751
Epoch 6/80
882/882 [==============================] - 0s 61us/step - loss: 1.2029 - acc: 0.4819
Epoch 7/80
882/882 [==============================] - 0s 62us/step - loss: 1.1633 - acc: 0.4909
Epoch 8/80
882/882 [==============================] - 0s 60us/step - loss: 1.1716 - acc: 0.4943
Epoch 9/80
882/882 [==============================] - 0s 62us/step - loss: 1.1537 - acc: 0.4977
Epoch 10/80
882/882 [==============================] - 0s 62us/step - loss: 1.1488 - acc: 0.5159
Epoch 11/80
882/882 [===================

Epoch 1/80
882/882 [==============================] - 5s 6ms/step - loss: 2.4547 - acc: 0.2177
Epoch 2/80
882/882 [==============================] - 0s 53us/step - loss: 1.6613 - acc: 0.3605
Epoch 3/80
882/882 [==============================] - 0s 43us/step - loss: 1.4925 - acc: 0.3980
Epoch 4/80
882/882 [==============================] - 0s 46us/step - loss: 1.3660 - acc: 0.4388
Epoch 5/80
882/882 [==============================] - ETA: 0s - loss: 1.2357 - acc: 0.453 - 0s 45us/step - loss: 1.2582 - acc: 0.4535
Epoch 6/80
882/882 [==============================] - 0s 50us/step - loss: 1.2277 - acc: 0.4773
Epoch 7/80
882/882 [==============================] - 0s 49us/step - loss: 1.1626 - acc: 0.5125
Epoch 8/80
882/882 [==============================] - 0s 44us/step - loss: 1.1101 - acc: 0.5488
Epoch 9/80
882/882 [==============================] - 0s 48us/step - loss: 1.0828 - acc: 0.5385
Epoch 10/80
882/882 [==============================] - 0s 44us/step - loss: 1.1043 - acc: 0.5272
Ep

Epoch 1/80
883/883 [==============================] - 5s 5ms/step - loss: 2.1441 - acc: 0.3046
Epoch 2/80
883/883 [==============================] - 0s 70us/step - loss: 1.4961 - acc: 0.4122
Epoch 3/80
883/883 [==============================] - 0s 61us/step - loss: 1.3766 - acc: 0.4304
Epoch 4/80
883/883 [==============================] - 0s 50us/step - loss: 1.3953 - acc: 0.4587
Epoch 5/80
883/883 [==============================] - 0s 51us/step - loss: 1.2685 - acc: 0.4926
Epoch 6/80
883/883 [==============================] - 0s 53us/step - loss: 1.3066 - acc: 0.4689
Epoch 7/80
883/883 [==============================] - 0s 67us/step - loss: 1.2689 - acc: 0.4643
Epoch 8/80
883/883 [==============================] - 0s 59us/step - loss: 1.2205 - acc: 0.4938
Epoch 9/80
883/883 [==============================] - 0s 58us/step - loss: 1.1679 - acc: 0.5142
Epoch 10/80
883/883 [==============================] - 0s 54us/step - loss: 1.2402 - acc: 0.4847
Epoch 11/80
883/883 [===================

Epoch 1/80
884/884 [==============================] - 4s 5ms/step - loss: 2.4295 - acc: 0.2059
Epoch 2/80
884/884 [==============================] - 0s 48us/step - loss: 1.8090 - acc: 0.3066
Epoch 3/80
884/884 [==============================] - 0s 66us/step - loss: 1.5459 - acc: 0.3948
Epoch 4/80
884/884 [==============================] - 0s 42us/step - loss: 1.4677 - acc: 0.4084
Epoch 5/80
884/884 [==============================] - 0s 42us/step - loss: 1.3325 - acc: 0.4593
Epoch 6/80
884/884 [==============================] - 0s 53us/step - loss: 1.3893 - acc: 0.4084
Epoch 7/80
884/884 [==============================] - 0s 81us/step - loss: 1.2357 - acc: 0.4649
Epoch 8/80
884/884 [==============================] - 0s 66us/step - loss: 1.2556 - acc: 0.4955
Epoch 9/80
884/884 [==============================] - 0s 71us/step - loss: 1.2302 - acc: 0.4808
Epoch 10/80
884/884 [==============================] - 0s 80us/step - loss: 1.2220 - acc: 0.5079
Epoch 11/80
884/884 [===================

Epoch 1/80
886/886 [==============================] - 6s 7ms/step - loss: 2.0790 - acc: 0.2517
Epoch 2/80
886/886 [==============================] - 0s 108us/step - loss: 1.5846 - acc: 0.3781
Epoch 3/80
886/886 [==============================] - 0s 106us/step - loss: 1.4325 - acc: 0.4300
Epoch 4/80
886/886 [==============================] - 0s 88us/step - loss: 1.3382 - acc: 0.4402
Epoch 5/80
886/886 [==============================] - 0s 121us/step - loss: 1.2546 - acc: 0.4921
Epoch 6/80
886/886 [==============================] - 0s 117us/step - loss: 1.1657 - acc: 0.5023
Epoch 7/80
886/886 [==============================] - 0s 91us/step - loss: 1.1513 - acc: 0.5135
Epoch 8/80
886/886 [==============================] - 0s 79us/step - loss: 1.1801 - acc: 0.4910
Epoch 9/80
886/886 [==============================] - 0s 63us/step - loss: 1.1582 - acc: 0.5034
Epoch 10/80
886/886 [==============================] - 0s 106us/step - loss: 1.1463 - acc: 0.5023
Epoch 11/80
886/886 [==============

Epoch 1/80
888/888 [==============================] - 6s 6ms/step - loss: 2.4349 - acc: 0.2185
Epoch 2/80
888/888 [==============================] - 0s 75us/step - loss: 1.8367 - acc: 0.3288
Epoch 3/80
888/888 [==============================] - 0s 62us/step - loss: 1.5771 - acc: 0.3626
Epoch 4/80
888/888 [==============================] - 0s 62us/step - loss: 1.5324 - acc: 0.3660
Epoch 5/80
888/888 [==============================] - 0s 60us/step - loss: 1.4476 - acc: 0.4020
Epoch 6/80
888/888 [==============================] - 0s 61us/step - loss: 1.3952 - acc: 0.4189
Epoch 7/80
888/888 [==============================] - 0s 62us/step - loss: 1.3027 - acc: 0.4730
Epoch 8/80
888/888 [==============================] - 0s 59us/step - loss: 1.3935 - acc: 0.4155
Epoch 9/80
888/888 [==============================] - 0s 60us/step - loss: 1.4445 - acc: 0.4099
Epoch 10/80
888/888 [==============================] - 0s 60us/step - loss: 1.4949 - acc: 0.4099
Epoch 11/80
888/888 [===================

In [222]:
train_test_model(lr=0.07, epochs=100, batch_size=64)

Epoch 1/100
877/877 [==============================] - 5s 5ms/step - loss: 2.4863 - acc: 0.2007
Epoch 2/100
877/877 [==============================] - 0s 44us/step - loss: 1.7511 - acc: 0.3352
Epoch 3/100
877/877 [==============================] - 0s 42us/step - loss: 1.3815 - acc: 0.4584
Epoch 4/100
877/877 [==============================] - 0s 46us/step - loss: 1.2824 - acc: 0.4766
Epoch 5/100
877/877 [==============================] - 0s 47us/step - loss: 1.2620 - acc: 0.4584
Epoch 6/100
877/877 [==============================] - 0s 56us/step - loss: 1.2270 - acc: 0.4846
Epoch 7/100
877/877 [==============================] - 0s 43us/step - loss: 1.2145 - acc: 0.4971
Epoch 8/100
877/877 [==============================] - 0s 46us/step - loss: 1.1702 - acc: 0.5051
Epoch 9/100
877/877 [==============================] - 0s 46us/step - loss: 1.2378 - acc: 0.4709
Epoch 10/100
877/877 [==============================] - 0s 43us/step - loss: 1.2043 - acc: 0.4983
Epoch 11/100
877/877 [========

877/877 [==============================] - 0s 42us/step - loss: 1.0223 - acc: 0.6009
Epoch 62/100
877/877 [==============================] - 0s 86us/step - loss: 0.9917 - acc: 0.6214
Epoch 63/100
877/877 [==============================] - 0s 42us/step - loss: 0.9934 - acc: 0.6089
Epoch 64/100
877/877 [==============================] - 0s 42us/step - loss: 0.9426 - acc: 0.6135
Epoch 65/100
877/877 [==============================] - 0s 46us/step - loss: 0.9718 - acc: 0.6157
Epoch 66/100
877/877 [==============================] - 0s 91us/step - loss: 0.9989 - acc: 0.6169
Epoch 67/100
877/877 [==============================] - 0s 114us/step - loss: 1.0858 - acc: 0.5792
Epoch 68/100
877/877 [==============================] - 0s 56us/step - loss: 1.0676 - acc: 0.5895
Epoch 69/100
877/877 [==============================] - 0s 51us/step - loss: 1.0418 - acc: 0.5872
Epoch 70/100
877/877 [==============================] - 0s 57us/step - loss: 1.0790 - acc: 0.5781
Epoch 71/100
877/877 [==========

880/880 [==============================] - 0s 52us/step - loss: 2.0762 - acc: 0.2170
Epoch 39/100
880/880 [==============================] - 0s 51us/step - loss: 2.0780 - acc: 0.2170
Epoch 40/100
880/880 [==============================] - 0s 48us/step - loss: 2.0811 - acc: 0.2000
Epoch 41/100
880/880 [==============================] - 0s 50us/step - loss: 2.0797 - acc: 0.1977
Epoch 42/100
880/880 [==============================] - 0s 41us/step - loss: 2.0766 - acc: 0.2125
Epoch 43/100
880/880 [==============================] - 0s 39us/step - loss: 2.0775 - acc: 0.2170
Epoch 44/100
880/880 [==============================] - 0s 39us/step - loss: 2.0815 - acc: 0.1920
Epoch 45/100
880/880 [==============================] - 0s 39us/step - loss: 2.0771 - acc: 0.2011
Epoch 46/100
880/880 [==============================] - 0s 47us/step - loss: 2.0764 - acc: 0.2216
Epoch 47/100
880/880 [==============================] - 0s 41us/step - loss: 2.0791 - acc: 0.2170
Epoch 48/100
880/880 [===========

881/881 [==============================] - 0s 75us/step - loss: 0.9594 - acc: 0.6050
Epoch 100/100
881/881 [==============================] - 0s 75us/step - loss: 0.9501 - acc: 0.5868
[1 7 1 4 6 4 5 6 5 3 2 5 6 3 4 2 4 5 1 3 4 5 3 3 2 3 3 3 2 2 5 4 3 3 4 1 1
 6 6 4 2 3 3 6 6 2 5 2 6 5 4 2 1 2 2 4 7 3 3 4 2 7 6 2 3 2 7 3 1 2 5 5 4 3
 3 5 7 2 3 3 2 7 6 4 3 8 4 6 3 2 2 3 3 4 1 5 2 5 2]
[ 1  5  1  5  7  4  4  6  7  4  2  4  6  2  3  1  3  7  0  4  4  6  4  2
  2  2  4  3  2  0  5  5  3  4  3  3  2  5  5  4  2  3  3  6  6  2  5  0
  5  7  3  1  0  2  2  5  8  2  2  4  1  8  7  3  4  3  9  4  1  1  4  6
  4  2  2  4 10  2  3  4  3  9  5  3  3  8  4  6  2  2  3  3  4  4  2  3
  2  4  3]
[0.35353535 0.88888889]
Epoch 1/100
882/882 [==============================] - 5s 6ms/step - loss: 2.5154 - acc: 0.1859
Epoch 2/100
882/882 [==============================] - 0s 92us/step - loss: 1.8506 - acc: 0.3209
Epoch 3/100
882/882 [==============================] - 0s 83us/step - loss: 1.6643 - acc: 0.36

882/882 [==============================] - 0s 83us/step - loss: 1.0698 - acc: 0.5601
Epoch 55/100
882/882 [==============================] - 0s 65us/step - loss: 1.0995 - acc: 0.5442
Epoch 56/100
882/882 [==============================] - 0s 69us/step - loss: 1.0844 - acc: 0.5578
Epoch 57/100
882/882 [==============================] - 0s 77us/step - loss: 1.1337 - acc: 0.5283
Epoch 58/100
882/882 [==============================] - 0s 70us/step - loss: 1.0900 - acc: 0.5522
Epoch 59/100
882/882 [==============================] - 0s 75us/step - loss: 1.0598 - acc: 0.5544
Epoch 60/100
882/882 [==============================] - 0s 68us/step - loss: 1.0714 - acc: 0.5476
Epoch 61/100
882/882 [==============================] - 0s 63us/step - loss: 1.0880 - acc: 0.5363
Epoch 62/100
882/882 [==============================] - 0s 61us/step - loss: 1.0996 - acc: 0.5601
Epoch 63/100
882/882 [==============================] - 0s 58us/step - loss: 1.0680 - acc: 0.5635
Epoch 64/100
882/882 [===========

884/884 [==============================] - 0s 58us/step - loss: 1.3264 - acc: 0.4581
Epoch 11/100
884/884 [==============================] - 0s 63us/step - loss: 1.3499 - acc: 0.4265
Epoch 12/100
884/884 [==============================] - 0s 53us/step - loss: 1.2573 - acc: 0.4649
Epoch 13/100
884/884 [==============================] - 0s 55us/step - loss: 1.2409 - acc: 0.4898
Epoch 14/100
884/884 [==============================] - 0s 100us/step - loss: 1.2492 - acc: 0.4977
Epoch 15/100
884/884 [==============================] - 0s 173us/step - loss: 1.2548 - acc: 0.4898
Epoch 16/100
884/884 [==============================] - 0s 124us/step - loss: 1.2314 - acc: 0.4729
Epoch 17/100
884/884 [==============================] - 0s 61us/step - loss: 1.1874 - acc: 0.5057
Epoch 18/100
884/884 [==============================] - 0s 55us/step - loss: 1.2433 - acc: 0.4762
Epoch 19/100
884/884 [==============================] - 0s 53us/step - loss: 1.2457 - acc: 0.4785
Epoch 20/100
884/884 [========

886/886 [==============================] - 0s 61us/step - loss: 1.0558 - acc: 0.5519
Epoch 72/100
886/886 [==============================] - 0s 60us/step - loss: 1.1068 - acc: 0.5113
Epoch 73/100
886/886 [==============================] - 0s 58us/step - loss: 1.0527 - acc: 0.5474
Epoch 74/100
886/886 [==============================] - 0s 63us/step - loss: 1.0844 - acc: 0.5384
Epoch 75/100
886/886 [==============================] - 0s 64us/step - loss: 1.1112 - acc: 0.5542
Epoch 76/100
886/886 [==============================] - 0s 61us/step - loss: 1.0972 - acc: 0.5384
Epoch 77/100
886/886 [==============================] - 0s 63us/step - loss: 1.1295 - acc: 0.5485
Epoch 78/100
886/886 [==============================] - 0s 88us/step - loss: 1.0942 - acc: 0.5339
Epoch 79/100
886/886 [==============================] - 0s 111us/step - loss: 1.0337 - acc: 0.5587
Epoch 80/100
886/886 [==============================] - 0s 96us/step - loss: 1.0261 - acc: 0.5700
Epoch 81/100
886/886 [==========

In [225]:
train_test_model(lr=0.05, epochs=80, batch_size=64)

Epoch 1/80
877/877 [==============================] - 7s 8ms/step - loss: 2.2672 - acc: 0.2497
Epoch 2/80
877/877 [==============================] - 0s 228us/step - loss: 1.5115 - acc: 0.3900
Epoch 3/80
877/877 [==============================] - 0s 99us/step - loss: 1.2924 - acc: 0.4470
Epoch 4/80
877/877 [==============================] - 0s 120us/step - loss: 1.2871 - acc: 0.4778
Epoch 5/80
877/877 [==============================] - 0s 119us/step - loss: 1.2480 - acc: 0.4675
Epoch 6/80
877/877 [==============================] - 0s 119us/step - loss: 1.1685 - acc: 0.5017
Epoch 7/80
877/877 [==============================] - 0s 105us/step - loss: 1.1587 - acc: 0.5165
Epoch 8/80
877/877 [==============================] - 0s 103us/step - loss: 1.1312 - acc: 0.5291
Epoch 9/80
877/877 [==============================] - 0s 65us/step - loss: 1.1225 - acc: 0.5051
Epoch 10/80
877/877 [==============================] - 0s 123us/step - loss: 1.1434 - acc: 0.5017
Epoch 11/80
877/877 [============

Epoch 1/80
877/877 [==============================] - 7s 8ms/step - loss: 2.3019 - acc: 0.2201
Epoch 2/80
877/877 [==============================] - 0s 82us/step - loss: 1.5466 - acc: 0.3991
Epoch 3/80
877/877 [==============================] - 0s 104us/step - loss: 1.3968 - acc: 0.4367
Epoch 4/80
877/877 [==============================] - 0s 82us/step - loss: 1.3138 - acc: 0.4607
Epoch 5/80
877/877 [==============================] - 0s 99us/step - loss: 1.2943 - acc: 0.4732
Epoch 6/80
877/877 [==============================] - 0s 98us/step - loss: 1.2650 - acc: 0.4641
Epoch 7/80
877/877 [==============================] - 0s 96us/step - loss: 1.1722 - acc: 0.4994
Epoch 8/80
877/877 [==============================] - 0s 108us/step - loss: 1.2724 - acc: 0.4835
Epoch 9/80
877/877 [==============================] - 0s 91us/step - loss: 1.1813 - acc: 0.4937
Epoch 10/80
877/877 [==============================] - 0s 99us/step - loss: 1.1555 - acc: 0.5188
Epoch 11/80
877/877 [=================

Epoch 1/80
880/880 [==============================] - 7s 9ms/step - loss: 1.9728 - acc: 0.3000
Epoch 2/80
880/880 [==============================] - 0s 75us/step - loss: 1.4873 - acc: 0.4159
Epoch 3/80
880/880 [==============================] - 0s 83us/step - loss: 1.2817 - acc: 0.4409
Epoch 4/80
880/880 [==============================] - 0s 53us/step - loss: 1.2844 - acc: 0.4830
Epoch 5/80
880/880 [==============================] - 0s 53us/step - loss: 1.1876 - acc: 0.4989
Epoch 6/80
880/880 [==============================] - 0s 45us/step - loss: 1.1842 - acc: 0.4966
Epoch 7/80
880/880 [==============================] - 0s 49us/step - loss: 1.2382 - acc: 0.5034
Epoch 8/80
880/880 [==============================] - 0s 48us/step - loss: 1.1774 - acc: 0.4841
Epoch 9/80
880/880 [==============================] - 0s 49us/step - loss: 1.1330 - acc: 0.4898
Epoch 10/80
880/880 [==============================] - 0s 51us/step - loss: 1.0976 - acc: 0.5386
Epoch 11/80
880/880 [===================

Epoch 1/80
881/881 [==============================] - 6s 7ms/step - loss: 2.3158 - acc: 0.2293
Epoch 2/80
881/881 [==============================] - 0s 96us/step - loss: 1.6177 - acc: 0.3859
Epoch 3/80
881/881 [==============================] - 0s 79us/step - loss: 1.4533 - acc: 0.4120
Epoch 4/80
881/881 [==============================] - 0s 78us/step - loss: 1.4529 - acc: 0.4234
Epoch 5/80
881/881 [==============================] - 0s 59us/step - loss: 1.3444 - acc: 0.4325
Epoch 6/80
881/881 [==============================] - 0s 94us/step - loss: 1.2599 - acc: 0.4665
Epoch 7/80
881/881 [==============================] - 0s 101us/step - loss: 1.2227 - acc: 0.5017
Epoch 8/80
881/881 [==============================] - 0s 96us/step - loss: 1.1989 - acc: 0.4711
Epoch 9/80
881/881 [==============================] - 0s 95us/step - loss: 1.1628 - acc: 0.5221
Epoch 10/80
881/881 [==============================] - 0s 81us/step - loss: 1.1814 - acc: 0.4858
Epoch 11/80
881/881 [==================

Epoch 1/80
882/882 [==============================] - 7s 7ms/step - loss: 2.0574 - acc: 0.2868
Epoch 2/80
882/882 [==============================] - 0s 88us/step - loss: 1.5042 - acc: 0.3776
Epoch 3/80
882/882 [==============================] - 0s 71us/step - loss: 1.3387 - acc: 0.4365
Epoch 4/80
882/882 [==============================] - 0s 69us/step - loss: 1.2342 - acc: 0.4841
Epoch 5/80
882/882 [==============================] - 0s 58us/step - loss: 1.1933 - acc: 0.4728
Epoch 6/80
882/882 [==============================] - 0s 56us/step - loss: 1.1832 - acc: 0.5113
Epoch 7/80
882/882 [==============================] - 0s 53us/step - loss: 1.1875 - acc: 0.5000
Epoch 8/80
882/882 [==============================] - 0s 54us/step - loss: 1.1589 - acc: 0.4932
Epoch 9/80
882/882 [==============================] - 0s 118us/step - loss: 1.1323 - acc: 0.5295
Epoch 10/80
882/882 [==============================] - 0s 91us/step - loss: 1.1321 - acc: 0.5068
Epoch 11/80
882/882 [==================

Epoch 1/80
882/882 [==============================] - 7s 8ms/step - loss: 2.2713 - acc: 0.2166
Epoch 2/80
882/882 [==============================] - 0s 116us/step - loss: 1.7467 - acc: 0.3673
Epoch 3/80
882/882 [==============================] - 0s 99us/step - loss: 1.5003 - acc: 0.3832
Epoch 4/80
882/882 [==============================] - 0s 115us/step - loss: 1.3446 - acc: 0.4399
Epoch 5/80
882/882 [==============================] - 0s 146us/step - loss: 1.2184 - acc: 0.4739
Epoch 6/80
882/882 [==============================] - 0s 107us/step - loss: 1.1609 - acc: 0.5102
Epoch 7/80
882/882 [==============================] - 0s 108us/step - loss: 1.1245 - acc: 0.5125
Epoch 8/80
882/882 [==============================] - 0s 113us/step - loss: 1.1166 - acc: 0.5204
Epoch 9/80
882/882 [==============================] - 0s 109us/step - loss: 1.1062 - acc: 0.5442
Epoch 10/80
882/882 [==============================] - 0s 115us/step - loss: 1.0707 - acc: 0.5556
Epoch 11/80
882/882 [===========

Epoch 1/80
883/883 [==============================] - 135s 153ms/step - loss: 2.2211 - acc: 0.2016
Epoch 2/80
883/883 [==============================] - 0s 86us/step - loss: 1.5189 - acc: 0.3918
Epoch 3/80
883/883 [==============================] - 0s 211us/step - loss: 1.3810 - acc: 0.4179
Epoch 4/80
883/883 [==============================] - 0s 74us/step - loss: 1.2771 - acc: 0.4439
Epoch 5/80
883/883 [==============================] - 0s 50us/step - loss: 1.2155 - acc: 0.4858
Epoch 6/80
883/883 [==============================] - 0s 53us/step - loss: 1.1950 - acc: 0.4745
Epoch 7/80
883/883 [==============================] - 0s 106us/step - loss: 1.2049 - acc: 0.4836
Epoch 8/80
883/883 [==============================] - 0s 86us/step - loss: 1.1893 - acc: 0.4881
Epoch 9/80
883/883 [==============================] - 0s 51us/step - loss: 1.1348 - acc: 0.5142
Epoch 10/80
883/883 [==============================] - 0s 51us/step - loss: 1.0941 - acc: 0.5379
Epoch 11/80
883/883 [=============

Epoch 1/80
884/884 [==============================] - 9s 10ms/step - loss: 1.9681 - acc: 0.3032
Epoch 2/80
884/884 [==============================] - 0s 120us/step - loss: 1.4650 - acc: 0.4208
Epoch 3/80
884/884 [==============================] - 0s 156us/step - loss: 1.3443 - acc: 0.4695
Epoch 4/80
884/884 [==============================] - 0s 93us/step - loss: 1.2216 - acc: 0.5068
Epoch 5/80
884/884 [==============================] - 0s 71us/step - loss: 1.2413 - acc: 0.4796
Epoch 6/80
884/884 [==============================] - 0s 61us/step - loss: 1.2350 - acc: 0.4774
Epoch 7/80
884/884 [==============================] - 0s 58us/step - loss: 1.1967 - acc: 0.5034
Epoch 8/80
884/884 [==============================] - 0s 58us/step - loss: 1.1560 - acc: 0.5124
Epoch 9/80
884/884 [==============================] - 0s 88us/step - loss: 1.1430 - acc: 0.5068
Epoch 10/80
884/884 [==============================] - 0s 64us/step - loss: 1.1353 - acc: 0.5192
Epoch 11/80
884/884 [================

Epoch 1/80
886/886 [==============================] - 16s 19ms/step - loss: 2.2239 - acc: 0.2393
Epoch 2/80
886/886 [==============================] - 0s 142us/step - loss: 1.5103 - acc: 0.3883
Epoch 3/80
886/886 [==============================] - 0s 286us/step - loss: 1.3049 - acc: 0.4515
Epoch 4/80
886/886 [==============================] - 0s 133us/step - loss: 1.2222 - acc: 0.4695
Epoch 5/80
886/886 [==============================] - 0s 121us/step - loss: 1.1686 - acc: 0.5113
Epoch 6/80
886/886 [==============================] - 0s 240us/step - loss: 1.1971 - acc: 0.5169
Epoch 7/80
886/886 [==============================] - 0s 123us/step - loss: 1.1704 - acc: 0.5147
Epoch 8/80
886/886 [==============================] - 0s 193us/step - loss: 1.1819 - acc: 0.4887
Epoch 9/80
886/886 [==============================] - 0s 99us/step - loss: 1.1289 - acc: 0.5068
Epoch 10/80
886/886 [==============================] - 0s 70us/step - loss: 1.0780 - acc: 0.5305
Epoch 11/80
886/886 [==========

Epoch 1/80
888/888 [==============================] - 9s 10ms/step - loss: 2.4519 - acc: 0.2016
Epoch 2/80
888/888 [==============================] - 0s 152us/step - loss: 1.9585 - acc: 0.2714
Epoch 3/80
888/888 [==============================] - 0s 238us/step - loss: 1.6784 - acc: 0.3795
Epoch 4/80
888/888 [==============================] - 0s 115us/step - loss: 1.4258 - acc: 0.4077
Epoch 5/80
888/888 [==============================] - 0s 118us/step - loss: 1.2945 - acc: 0.4550
Epoch 6/80
888/888 [==============================] - 0s 114us/step - loss: 1.2874 - acc: 0.4673
Epoch 7/80
888/888 [==============================] - 0s 98us/step - loss: 1.2056 - acc: 0.4865
Epoch 8/80
888/888 [==============================] - 0s 87us/step - loss: 1.2055 - acc: 0.5011
Epoch 9/80
888/888 [==============================] - 0s 84us/step - loss: 1.1606 - acc: 0.5203
Epoch 10/80
888/888 [==============================] - 0s 114us/step - loss: 1.1573 - acc: 0.5090
Epoch 11/80
888/888 [============

In [226]:
train_test_model(lr=0.01, epochs=30, batch_size=64)

Epoch 1/30
877/877 [==============================] - 8s 9ms/step - loss: 1.9897 - acc: 0.2851
Epoch 2/30
877/877 [==============================] - 0s 192us/step - loss: 1.4704 - acc: 0.4253
Epoch 3/30
877/877 [==============================] - 0s 117us/step - loss: 1.2139 - acc: 0.4914
Epoch 4/30
877/877 [==============================] - 0s 113us/step - loss: 1.1611 - acc: 0.5154
Epoch 5/30
877/877 [==============================] - 0s 116us/step - loss: 1.0751 - acc: 0.5564
Epoch 6/30
877/877 [==============================] - 0s 105us/step - loss: 1.0485 - acc: 0.5542
Epoch 7/30
877/877 [==============================] - 0s 90us/step - loss: 1.0142 - acc: 0.5713
Epoch 8/30
877/877 [==============================] - 0s 154us/step - loss: 0.9256 - acc: 0.6249
Epoch 9/30
877/877 [==============================] - 0s 174us/step - loss: 0.8868 - acc: 0.6420
Epoch 10/30
877/877 [==============================] - 0s 106us/step - loss: 0.8485 - acc: 0.6442
Epoch 11/30
877/877 [===========

882/882 [==============================] - 0s 61us/step - loss: 0.4615 - acc: 0.8175
Epoch 23/30
882/882 [==============================] - 0s 62us/step - loss: 0.4003 - acc: 0.8447
Epoch 24/30
882/882 [==============================] - 0s 60us/step - loss: 0.3786 - acc: 0.8537
Epoch 25/30
882/882 [==============================] - 0s 63us/step - loss: 0.4226 - acc: 0.8333
Epoch 26/30
882/882 [==============================] - 0s 59us/step - loss: 0.3671 - acc: 0.8651
Epoch 27/30
882/882 [==============================] - 0s 69us/step - loss: 0.3594 - acc: 0.8628
Epoch 28/30
882/882 [==============================] - 0s 81us/step - loss: 0.3572 - acc: 0.8571
Epoch 29/30
882/882 [==============================] - 0s 83us/step - loss: 0.3649 - acc: 0.8583
Epoch 30/30
882/882 [==============================] - 0s 73us/step - loss: 0.3714 - acc: 0.8537
[ 1  2  0  5 10  4  1  5  2  9  6  2  2  3  4  4  0  4  3  2  5  3  2  6
  1  3  3  5  3  9  5  5  3  5  5  2  3  3  3  4  4  3  3  3  4  5

Epoch 1/30
884/884 [==============================] - 7s 8ms/step - loss: 1.9464 - acc: 0.2862
Epoch 2/30
884/884 [==============================] - 0s 97us/step - loss: 1.4454 - acc: 0.4152
Epoch 3/30
884/884 [==============================] - 0s 95us/step - loss: 1.2133 - acc: 0.4921
Epoch 4/30
884/884 [==============================] - 0s 84us/step - loss: 1.1341 - acc: 0.5045
Epoch 5/30
884/884 [==============================] - 0s 109us/step - loss: 1.0991 - acc: 0.5317
Epoch 6/30
884/884 [==============================] - 0s 89us/step - loss: 1.0722 - acc: 0.5385
Epoch 7/30
884/884 [==============================] - 0s 91us/step - loss: 1.0232 - acc: 0.5724
Epoch 8/30
884/884 [==============================] - 0s 95us/step - loss: 0.9911 - acc: 0.5735
Epoch 9/30
884/884 [==============================] - 0s 92us/step - loss: 0.9661 - acc: 0.5871
Epoch 10/30
884/884 [==============================] - 0s 104us/step - loss: 0.8943 - acc: 0.6210
Epoch 11/30
884/884 [=================

In [227]:
train_test_model(lr=0.01, epochs=60, batch_size=64)

Epoch 1/60
877/877 [==============================] - 7s 8ms/step - loss: 1.9901 - acc: 0.2566
Epoch 2/60
877/877 [==============================] - 0s 125us/step - loss: 1.4592 - acc: 0.4447
Epoch 3/60
877/877 [==============================] - 0s 106us/step - loss: 1.2583 - acc: 0.5051
Epoch 4/60
877/877 [==============================] - 0s 106us/step - loss: 1.1870 - acc: 0.5029
Epoch 5/60
877/877 [==============================] - 0s 107us/step - loss: 1.1017 - acc: 0.5416
Epoch 6/60
877/877 [==============================] - 0s 101us/step - loss: 1.0589 - acc: 0.5507
Epoch 7/60
877/877 [==============================] - 0s 105us/step - loss: 1.0140 - acc: 0.5678
Epoch 8/60
877/877 [==============================] - 0s 97us/step - loss: 0.9326 - acc: 0.6123
Epoch 9/60
877/877 [==============================] - 0s 113us/step - loss: 0.9228 - acc: 0.6100
Epoch 10/60
877/877 [==============================] - 0s 116us/step - loss: 0.9067 - acc: 0.6123
Epoch 11/60
877/877 [===========

880/880 [==============================] - 0s 68us/step - loss: 0.1961 - acc: 0.9250
Epoch 37/60
880/880 [==============================] - 0s 68us/step - loss: 0.2212 - acc: 0.9193
Epoch 38/60
880/880 [==============================] - 0s 67us/step - loss: 0.3250 - acc: 0.8932
Epoch 39/60
880/880 [==============================] - 0s 80us/step - loss: 0.2477 - acc: 0.8909
Epoch 40/60
880/880 [==============================] - 0s 77us/step - loss: 0.2339 - acc: 0.9148
Epoch 41/60
880/880 [==============================] - 0s 89us/step - loss: 0.2493 - acc: 0.9148
Epoch 42/60
880/880 [==============================] - 0s 75us/step - loss: 0.1788 - acc: 0.9364
Epoch 43/60
880/880 [==============================] - 0s 72us/step - loss: 0.1718 - acc: 0.9284
Epoch 44/60
880/880 [==============================] - 0s 74us/step - loss: 0.1862 - acc: 0.9375
Epoch 45/60
880/880 [==============================] - 0s 72us/step - loss: 0.3035 - acc: 0.8943
Epoch 46/60
880/880 [=====================

882/882 [==============================] - 0s 92us/step - loss: 1.0968 - acc: 0.5340
Epoch 6/60
882/882 [==============================] - 0s 77us/step - loss: 1.0134 - acc: 0.5748
Epoch 7/60
882/882 [==============================] - 0s 86us/step - loss: 0.9892 - acc: 0.5828
Epoch 8/60
882/882 [==============================] - 0s 100us/step - loss: 0.9780 - acc: 0.5907
Epoch 9/60
882/882 [==============================] - 0s 130us/step - loss: 0.8874 - acc: 0.6361
Epoch 10/60
882/882 [==============================] - 0s 102us/step - loss: 0.8764 - acc: 0.6576
Epoch 11/60
882/882 [==============================] - 0s 91us/step - loss: 0.8091 - acc: 0.6678
Epoch 12/60
882/882 [==============================] - 0s 74us/step - loss: 0.7899 - acc: 0.6746
Epoch 13/60
882/882 [==============================] - 0s 75us/step - loss: 0.7320 - acc: 0.7041
Epoch 14/60
882/882 [==============================] - 0s 91us/step - loss: 0.7320 - acc: 0.7029
Epoch 15/60
882/882 [======================

884/884 [==============================] - 0s 79us/step - loss: 0.1634 - acc: 0.9502
Epoch 41/60
884/884 [==============================] - 0s 71us/step - loss: 0.1687 - acc: 0.9367
Epoch 42/60
884/884 [==============================] - 0s 84us/step - loss: 0.1949 - acc: 0.9321
Epoch 43/60
884/884 [==============================] - 0s 85us/step - loss: 0.2838 - acc: 0.9005
Epoch 44/60
884/884 [==============================] - 0s 101us/step - loss: 0.3652 - acc: 0.8643
Epoch 45/60
884/884 [==============================] - 0s 83us/step - loss: 0.3610 - acc: 0.8676
Epoch 46/60
884/884 [==============================] - 0s 85us/step - loss: 0.2466 - acc: 0.9061
Epoch 47/60
884/884 [==============================] - 0s 86us/step - loss: 0.2096 - acc: 0.9253
Epoch 48/60
884/884 [==============================] - 0s 83us/step - loss: 0.2105 - acc: 0.9333
Epoch 49/60
884/884 [==============================] - 0s 75us/step - loss: 0.1831 - acc: 0.9457
Epoch 50/60
884/884 [====================